In [1]:
import requests
import pandas as pd
from datetime import datetime, timezone
import os
from dotenv import load_dotenv

# Load API key
load_dotenv()
COINGECKO_API_KEY = os.getenv("GECKO_KEY")

def fetch_price_history(days=1500, coin_ids=["bitcoin", "ethereum", "tether", "usd-coin"]):
    rows = []
    headers = {"x-cg-pro-api-key": COINGECKO_API_KEY}

    for coin in coin_ids:
        url = f"https://pro-api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {"vs_currency": "usd", "days": days}

        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()

        for i, (ts, price) in enumerate(data["prices"]):
            timestamp = datetime.fromtimestamp(ts / 1000, tz=timezone.utc)
            market_cap = data["market_caps"][i][1] if i < len(data["market_caps"]) else None
            volume_24h = data["total_volumes"][i][1] if i < len(data["total_volumes"]) else None

            rows.append({
                "timestamp": timestamp,
                "asset": coin.lower(),
                "price_usd": price,
                "market_cap": market_cap,
                "volume_24h": volume_24h
            })

    df = pd.DataFrame(rows)

    # Resample to hourly frequency per asset
    df = (
        df.set_index("timestamp")
          .groupby("asset", group_keys=False)
          .apply(lambda g: g.resample("1h").ffill())  # forward-fill
          .reset_index()
    )

    # Print min/max timestamp per asset for debugging
    for asset in df['asset'].unique():
        asset_df = df[df['asset'] == asset]
        print(f"{asset}: min={asset_df['timestamp'].min()}, max={asset_df['timestamp'].max()}")

    return df

# Example: fetch enough days to cover all summary weeks
prices_df = fetch_price_history(days=1500)
print(prices_df.head(10))
print(prices_df.tail(10))

C:\Users\Muhammad\AppData\Local\Temp\ipykernel_12284\495625190.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.resample("1h").ffill())  # forward-fill


bitcoin: min=2021-08-12 00:00:00+00:00, max=2025-09-19 10:00:00+00:00
ethereum: min=2021-08-12 00:00:00+00:00, max=2025-09-19 10:00:00+00:00
tether: min=2021-08-12 00:00:00+00:00, max=2025-09-19 10:00:00+00:00
usd-coin: min=2021-08-12 00:00:00+00:00, max=2025-09-19 10:00:00+00:00
                  timestamp    asset     price_usd    market_cap    volume_24h
0 2021-08-12 00:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
1 2021-08-12 01:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
2 2021-08-12 02:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
3 2021-08-12 03:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
4 2021-08-12 04:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
5 2021-08-12 05:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
6 2021-08-12 06:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
7 2021-08-12 07:00:00+00:00  bitcoin  45652.060125  8.576606e+11  3.567983e+10
8 2021-0

In [2]:
import sqlite3

# Connect to DB
conn = sqlite3.connect("../data/blockchain.db")
cursor = conn.cursor()

# Drop old price table if it exists
cursor.execute("DROP TABLE IF EXISTS prices;")
conn.commit()

# Now save the fresh price data
prices_df.to_sql("prices", conn, if_exists="replace", index=False)

conn.close()
print("Old price table deleted and new one created ✅")

Old price table deleted and new one created ✅


In [4]:
# Connect to SQLite
conn = sqlite3.connect("../data/blockchain.db")
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    asset TEXT,
    price_usd REAL,
    market_cap REAL,
    volume_24h REAL
)
""")

# Fetch and insert
prices_df.to_sql("prices", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

print("✅ Data inserted into SQLite successfully!")

✅ Data inserted into SQLite successfully!


In [5]:
import sqlite3
def save_price(prices_df, db_path="../data/blockchain.db"):
    conn = sqlite3.connect(db_path)
    prices_df.to_sql("price", conn, if_exists="replace", index=False)
    #conn.close()
    print("✅ Price data saved to database")

save_price(prices_df)

✅ Price data saved to database


In [6]:
conn = sqlite3.connect("../data/blockchain.db")
df_check = pd.read_sql("SELECT * FROM prices LIMIT 10;", conn)
#conn.close()

print(df_check)

                   timestamp    asset     price_usd    market_cap  \
0  2021-08-12 00:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
1  2021-08-12 01:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
2  2021-08-12 02:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
3  2021-08-12 03:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
4  2021-08-12 04:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
5  2021-08-12 05:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
6  2021-08-12 06:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
7  2021-08-12 07:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
8  2021-08-12 08:00:00+00:00  bitcoin  45652.060125  8.576606e+11   
9  2021-08-12 09:00:00+00:00  bitcoin  45652.060125  8.576606e+11   

     volume_24h  
0  3.567983e+10  
1  3.567983e+10  
2  3.567983e+10  
3  3.567983e+10  
4  3.567983e+10  
5  3.567983e+10  
6  3.567983e+10  
7  3.567983e+10  
8  3.567983e+10  
9  3.567983e+10  
